In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [2]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")

In [ ]:
## GRAPH TRAVERSAL

In [4]:
gph_conn.query("""
               CREATE (FL:State {code: "FL", name: "Florida", population: 21500000})
               CREATE (AL:State {code: "AL", name: "Alabama", population: 4900000})
               CREATE (GA:State {code: "GA", name: "Georgia", population: 10600000})
               CREATE (MS:State {code: "MS", name: "Mississippi", population: 3000000})
               CREATE (TN:State {code: "TN", name: "Tennessee", population: 6800000})
               CREATE (NC:State {code: "NC", name: "North Carolina", population: 10500000})
               CREATE (SC:State {code: "SC", name: "South Carolina", population: 5100000})
               
               CREATE (FL)-[:SHARE_BORDER_WITH]->(AL) 
               CREATE (FL)-[:SHARE_BORDER_WITH]->(GA) 
               CREATE (AL)-[:SHARE_BORDER_WITH]->(MS) 
               CREATE (AL)-[:SHARE_BORDER_WITH]->(TN) 
               CREATE (GA)-[:SHARE_BORDER_WITH]->(AL) 
               CREATE (GA)-[:SHARE_BORDER_WITH]->(NC) 
               CREATE (GA)-[:SHARE_BORDER_WITH]->(SC) 
               CREATE (SC)-[:SHARE_BORDER_WITH]->(NC) 
               CREATE (TN)-[:SHARE_BORDER_WITH]->(MS) 
               CREATE (NC)-[:SHARE_BORDER_WITH]->(TN)
               """)

[]

In [5]:
gph_conn.query("""
               MATCH (:State {code: 'FL'})-[:SHARE_BORDER_WITH]-(n)
               RETURN n.name
               """)

[<Record n.name='Georgia'>, <Record n.name='Alabama'>]

In [6]:
gph_conn.query("""
               MATCH (:State {code: 'FL'})-[:SHARE_BORDER_WITH]-(n)
               RETURN n.name AS state_name, n.population AS state_population
               ORDER BY n.population DESC
               """)

[<Record state_name='Georgia' state_population=10600000>,
 <Record state_name='Alabama' state_population=4900000>]

In [ ]:
### 2-Hops away

In [7]:
gph_conn.query("""
               MATCH (:State {code: 'FL'})-[:SHARE_BORDER_WITH]-
               (neighbor)-[:SHARE_BORDER_WITH]-(n)
               RETURN n.name
               """)

[<Record n.name='Alabama'>,
 <Record n.name='North Carolina'>,
 <Record n.name='South Carolina'>,
 <Record n.name='Georgia'>,
 <Record n.name='Mississippi'>,
 <Record n.name='Tennessee'>]

In [10]:
gph_conn.query("""
               MATCH (FL:State {code: 'FL'})-[:SHARE_BORDER_WITH]-
               (neighbor)-[:SHARE_BORDER_WITH]-(n)
               WHERE NOT (FL)-[:SHARE_BORDER_WITH]-(n)
               RETURN n.name
               """)

[<Record n.name='North Carolina'>,
 <Record n.name='South Carolina'>,
 <Record n.name='Mississippi'>,
 <Record n.name='Tennessee'>]

In [11]:
gph_conn.query("""
               MATCH (:State {code: 'FL'})-[:SHARE_BORDER_WITH*2]-(n)
               RETURN n.name
               """)

[<Record n.name='Tennessee'>,
 <Record n.name='Mississippi'>,
 <Record n.name='Georgia'>,
 <Record n.name='South Carolina'>,
 <Record n.name='North Carolina'>,
 <Record n.name='Alabama'>]

In [12]:
gph_conn.query("""
               CREATE (:State {code: "AK", name: "Alaska", population: 700000})
               """)

[]

In [13]:
gph_conn.query("""
               MATCH (n:State {code: 'AK'})-[:SHARE_BORDER_WITH]-(m)
               RETURN n, m
               """)

[]

In [14]:
gph_conn.query("""
               MATCH (n:State {code: 'AK'})
               OPTIONAL MATCH (n)-[:SHARE_BORDER_WITH]-(m)
               RETURN n.name, m.name
               """)

[<Record n.name='Alaska' m.name=None>]

In [ ]:
## AGGREGATE

In [15]:
gph_conn.query("""
               MATCH (FL:State {code: 'FL'})-[:SHARE_BORDER_WITH]-(n)
               RETURN FL.name AS state_name, COUNT(n.code) AS nbr_cnt
               """)

[<Record state_name='Florida' nbr_cnt=2>]

In [17]:
gph_conn.query("""
               MATCH (s:State)-[:SHARE_BORDER_WITH]-(n)
               WITH s.name AS state_name
               ,    s.population AS state_population
               ,    toFloat(SUM(n.population)) AS nbr_population
               RETURN state_name, state_population, state_population / nbr_population AS rt
               """)

[<Record state_name='Florida' state_population=21500000 rt=1.3870967741935485>,
 <Record state_name='Alabama' state_population=4900000 rt=0.11694510739856802>,
 <Record state_name='Georgia' state_population=10600000 rt=0.2523809523809524>,
 <Record state_name='Mississippi' state_population=3000000 rt=0.2564102564102564>,
 <Record state_name='Tennessee' state_population=6800000 rt=0.3695652173913043>,
 <Record state_name='North Carolina' state_population=10500000 rt=0.4666666666666667>,
 <Record state_name='South Carolina' state_population=5100000 rt=0.24170616113744076>]

In [ ]:
## COLLECT & UNNEST

In [18]:
gph_conn.query("""
               MATCH (:State {code: 'FL'})-[:SHARE_BORDER_WITH]-(n)
               RETURN COLLECT(n.code)
               """)

[<Record COLLECT(n.code)=['GA', 'AL']>]

In [21]:
gph_conn.query("""
               MATCH (:State {code: 'FL'})-[:SHARE_BORDER_WITH]-(n)
               WITH COLLECT(n.code) AS codes
               UNWIND codes AS c
               RETURN c
               """)

[<Record c='GA'>, <Record c='AL'>]

In [ ]:
## LOAD DATA

In [27]:
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/PacktPublishing/Hands-On-Graph-Analytics-with-Neo4j/master/Chapter02/data/usa_state_neighbors_edges.csv' AS row
               FIELDTERMINATOR ';'
               MERGE (n:State {code: row.code})
               MERGE (m:State {code: row.neighbor_code})
               MERGE (n)-[:SHARE_BORDER_WITH]-(m)
               """)

[]

In [31]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[] relationships=[]>]

In [45]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [34]:
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/PacktPublishing/Hands-On-Graph-Analytics-with-Neo4j/master/Chapter02/data/usa_state_neighbors_all.csv' AS row
               FIELDTERMINATOR ';'
               WITH row.code AS state, SPLIT(row.neighbors, ',') AS nbrs
               MERGE (a:State {code: state})
               WITH a, nbrs
               UNWIND nbrs AS nbr
               WITH a, nbr
               WHERE nbr <> ''
               MERGE (b:State {code: nbr})
               MERGE (a)-[:SHARE_BORDER_WITH]-(b)
               """)

[]

In [43]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[] relationships=[]>]

In [44]:
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/PacktPublishing/Hands-On-Graph-Analytics-with-Neo4j/master/Chapter02/data/usa_state_neighbors_edges.csv' AS row
               FIELDTERMINATOR ';'
               MERGE (n:State {code: row.code})
               """)               
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/PacktPublishing/Hands-On-Graph-Analytics-with-Neo4j/master/Chapter02/data/usa_state_neighbors_edges.csv' AS row
               FIELDTERMINATOR ';'
               MERGE (m:State {code: row.neighbor_code})
               """)
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/PacktPublishing/Hands-On-Graph-Analytics-with-Neo4j/master/Chapter02/data/usa_state_neighbors_edges.csv' AS row
               FIELDTERMINATOR ';'
               MATCH (n:State {code: row.code})
               MATCH (m:State {code: row.neighbor_code})
               MERGE (n)-[:SHARE_BORDER_WITH]-(m)
               """)

[]

In [50]:
gph_conn.query("""
               dbms.security.procedures.whitelist= apoc.load.*
               CALL apoc.load.json('https://raw.githubusercontent.com/PacktPublishing/Hands-On-Graph-Analytics-with-Neo4j/master/Chapter02/data/neo4j_repos_github.json')
               YIELD value AS item
               RETURN item
               LIMIT 1
               """)

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'b': expected 'r/R' or 'e/E' (line 2, column 2 (offset: 17))
"dbms.security.procedures.whitelist= apoc.load.*;"
                  ^}
